In [1]:
'''
Author: DengRui
Date: 2023-09-11 02:10:21
LastEditors: DengRui
LastEditTime: 2023-09-11 02:20:39
FilePath: /DeepSub/dataprocess/get_train_test.ipynb
Description:  get train and test data
Copyright (c) 2023 by DengRui, All Rights Reserved. 
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('__file__')))
sys.path.append('./tool/')
from att import Attention
from keras.models import load_model

Using TensorFlow backend.


# data load

In [3]:
# load dataset and features
dataset  = pd.read_csv('./DATA/Dataset_0724.csv')
dataset = dataset.rename(columns={'Entry':'uniprot_id','Sequence':'seq'})
feature = pd.read_feather('./DATA/feature_esm2_20230911.feather')
dataset = dataset.merge(feature,on='uniprot_id',how='left')

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)
test_set = test_data.copy()
train_set = train_data.copy()

# baseline predict

### queen 

In [ ]:
# load model
model_location = "../model/queen/QUEEN_MLPmodel_final.pkl"
with open(model_location, "rb") as f:
  QUEEN_model = pickle.load(f)
  
# queen pred
y_test = QUEEN_model.predict(np.array(test_set.iloc[:,3:]))
inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}
y_test_transformed = np.array([inv_map[x] for x in y_test])
print("These are the predicted labels:")
print(y_test_transformed)

### ml

In [ ]:
# to do list

### deepsub

In [ ]:
model = load_model("../model/deepsub/deepsub.h5",custom_objects={"Attention": Attention},compile=False)
predicted = model.predict(np.array(test_data.iloc[:,3:]).reshape(test_data.shape[0],1,-1))
predicted_labels = np.argmax(predicted, axis=1)
label_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12}
y_test_transformed = [label_map[x] for x in predicted_labels]
print("These are the predicted labels:")
print(y_test_transformed)